# Analysis Starter Notebook
This notebook contains starter code for:

1. Importing a 3D height map of a glyph and computing its 2D spatial FFT to identify dominant spatial frequencies.
2. Running a scalar Fresnel diffraction simulation treating the height map as a phase mask.

(These are starter templates — adapt paths and parameters for your data.)

In [ ]:
# Required libraries
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# 1) Load heightmap (grayscale image where value -> height)
def load_heightmap(path):
    img = Image.open(path).convert('L')
    arr = np.array(img).astype(float)
    # normalize to meters assuming user provides scale
    return arr / 255.0

# 2) Compute 2D spatial FFT and show magnitude
def spatial_fft(heightmap):
    H = np.fft.fftshift(np.fft.fft2(heightmap))
    mag = np.abs(H)
    return mag

# 3) Simple Fresnel scalar propagation (FFT based)
def fresnel_propagation(field, wavelength, z, dx):
    # field: complex field at source plane (2D)
    # wavelength: meters, z: propagation distance meters, dx: sample spacing meters
    k = 2*np.pi / wavelength
    ny, nx = field.shape
    fx = np.fft.fftfreq(nx, d=dx)
    fy = np.fft.fftfreq(ny, d=dx)
    FX, FY = np.meshgrid(np.fft.fftshift(fx), np.fft.fftshift(fy))
    H = np.exp(-1j * np.pi * wavelength * z * (FX**2 + FY**2) * (2*np.pi)**2)
    F = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(field)))
    U2 = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(F * H)))
    return U2

# Example usage (replace 'heightmap.png' with your file)
# hm = load_heightmap('heightmap.png')
# plt.imshow(hm, cmap='gray'); plt.title('Heightmap'); plt.show()
# mag = spatial_fft(hm)
# plt.imshow(np.log1p(mag), cmap='inferno'); plt.title('Spatial FFT magnitude'); plt.show()

# To test Fresnel propagation you need to form a complex pupil field:
# wavelength = 532e-9
# dx = 10e-6  # sample spacing
# field = np.exp(1j * 2*np.pi * hm / wavelength)  # phase mask approximation
# z = 0.5  # meters
# U2 = fresnel_propagation(field, wavelength, z, dx)
# plt.imshow(np.abs(U2)); plt.title('Propagated intensity'); plt.show()


## Notes
- The Fresnel propagation function above is a simplified scalar approximation. For accurate EM scattering at optical wavelengths, use dedicated EM solvers.
- Scale (dx) and actual physical height mapping are critical — ensure the height map units are converted to meters before using.
